In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train.columns

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue'],
      dtype='object')

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [6]:
train['coll_id'] = train['belongs_to_collection'].fillna('[{"id":0}]').map(lambda x: (ast.literal_eval(x))[0]['id'])

In [7]:
dummies = pd.get_dummies(train['coll_id'],drop_first=True).values

In [8]:
reg = LinearRegression()

In [9]:
X = train[['budget','popularity']].values
y = train['revenue'].values

In [10]:
X = np.hstack((X,dummies))

In [11]:
reg.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [12]:
cross_val_score(reg,X,y,cv=10).mean()

0.637817742760891

Now try with the mean instead of dummies

In [13]:
colls = pd.read_csv('collections.csv',index_col=0,squeeze=True)
colls.head()

10     749699164
84     531269279
119    898827882
151     96733333
230    190916236
Name: average_revenue, dtype: int64

In [14]:
train['coll_rev'] = train['coll_id'].map(colls).fillna(0)

In [15]:
X_col = train[['budget','popularity','coll_rev']].values

In [16]:
reg.fit(X_col,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [17]:
cross_val_score(reg,X_col,y,cv=10).mean()

0.7616451754160671

Seems better. I will also try to add the std for each collection.

### Make prediction, just for fun

In [18]:
test['coll_id'] = test['belongs_to_collection'].fillna('[{"id":0}]').map(
    lambda x: (ast.literal_eval(x))[0]['id'])

In [19]:
test['coll_rev'] = test['coll_id'].map(colls).fillna(0)

In [21]:
X_test = test[['budget','popularity','coll_rev']].values

In [23]:
pred = reg.predict(X_test)

In [24]:
pred.shape

(4398,)

In [28]:
submit = pd.read_csv('sample_submission.csv',index_col='id')
submit.head()

,revenue
id,
3001,1000000
3002,1000000
3003,1000000
3004,1000000
3005,1000000


In [29]:
submit['revenue'] = pred

In [31]:
submit.head()

,revenue
id,
3001,4.393471e+07
3002,5.405677e+06
3003,1.279281e+07
3004,2.403209e+07
3005,7.758561e+06


In [32]:
submit.to_csv('bdgt_pop_coll_ave.csv')